In [ ]:
import pandas as pd
import numpy as np

importing master data

In [ ]:
dataset=pd.read_csv('/content/master recipe data.csv')
dataset

In [ ]:
dataset.isnull().sum()/len(dataset)*100

importing review data

In [ ]:
df_review=pd.read_csv('/content/master review data.csv')
df_review

merging

In [ ]:
df=pd.merge(dataset,df_review,on='RecipeId',how='inner')
df

In [ ]:
df.isnull().sum()

Sentiment Analysis

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax


In [ ]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)


sample

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
text = "Hello"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

In [ ]:
print('positive',scores[0])
print('neutral',scores[1])
print('negative',scores[2])

In [ ]:
scores_dict={'positive':[],'neutral':[],'negative':[]}

Looping through the data and finding sentiment scores for all

In [ ]:
for i in range(len(df['ReviewText'])):
  try:
    text=df['ReviewText'][i]
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict['positive'].append(scores[0])
    scores_dict['neutral'].append(scores[1])
    scores_dict['negative'].append(scores[2])
  except:
    print('failed for',df['RecipeId'][i])
    pass

In [ ]:
print(scores_dict)

In [ ]:
df.to_csv('merge.csv')

dropping rows which had too long reviews

In [ ]:
df['ReviewText'][2322]

In [ ]:
df['RecipeId'][2322]

In [ ]:
df['ReviewText'][3414]

In [ ]:
df['RecipeId'][3414]

In [ ]:
df['ReviewText'][7912]

In [ ]:
df['RecipeId'][7912]

In [ ]:
df['ReviewText'][18792]

In [ ]:
df['RecipeId'][18792]

In [ ]:
df['ReviewText'][18880]

In [ ]:
df['RecipeId'][18880]

In [ ]:
df.drop(df.index[[2322,3414,7912,18792,18880]],inplace=True)

In [ ]:
df.shape

In [ ]:
df1=df

##NOTE: POSITIVE AND NEGATIVE ARE SWAPPED HERE 

In [ ]:
print(len(scores_dict['positive']))

In [ ]:
df1['positive']=scores_dict['negative'][2322:25236]

In [ ]:
df1['negative']=scores_dict['positive'][2322:25236]

In [ ]:
df1['neutral']=scores_dict['neutral'][2322:25236]

In [ ]:
df1.head(10)

In [ ]:
l1=[]
for i in range(2322,25236):
  if(scores_dict['neutral'][i]>scores_dict['positive'][i] and scores_dict['neutral'][i]>scores_dict['negative'][i]):
    if(scores_dict['positive'][i]>scores_dict['negative'][i]):
      l1.append(2)
    if(scores_dict['negative'][i]>scores_dict['positive'][i]):
      l1.append(3)
    continue
  elif(scores_dict['positive'][i]>scores_dict['negative'][i]):
    if(scores_dict['positive'][i]>0.5):
      l1.append(1)
    elif(scores_dict['positive'][i]<0.5):
      l1.append(2)
    continue
  elif(scores_dict['negative'][i]>scores_dict['positive'][i]):
    if(scores_dict['negative'][i]>0.5):
      l1.append(5)
    elif(scores_dict['negative'][i]<0.5):
      l1.append(4)


In [ ]:
l1

In [ ]:
len(l1)

In [ ]:
df1['Predicted_rating']=l1

In [ ]:
df1.head(10)

In [ ]:
df2=df1[['RecipeId','Predicted_rating']].copy()
df2

In [ ]:
df2.to_csv('random_state42_submission.csv',index=False)

In [ ]:
l2=df1['RecipeId'].unique()

In [ ]:
df3=df2.groupby(['RecipeId']).mean().round().reset_index()

In [ ]:
df3

In [ ]:
df3.to_csv('random_state42_submission_codeicon.csv',index=False)

In [ ]:
df3

In [ ]:
df4=pd.merge(dataset,df3,on='RecipeId',how='inner')
df4

In [ ]:
df4.isnull().sum()

In [ ]:
df5=df4

In [ ]:
df5.drop(['Cuisine'], axis=1,inplace=True)

In [ ]:
df5.head(10)

In [ ]:
df5 = df5[df5['Calories'].notna()]

In [ ]:
df5.isnull().sum()

In [ ]:
df5['Meal_Type'].unique()

In [ ]:
df5['Dish'] = df5['Dish_Type'].fillna(df5['Dish_Type'].mode().iloc[0])

In [ ]:
df5

In [ ]:
df5.drop(['Dish_Type'], axis=1,inplace=True)

In [ ]:
df5.isnull().sum()

In [ ]:
df5['Flavor_Type'] = df5['Flavor'].fillna(df5['Flavor'].mode().iloc[0])

In [ ]:
df5.head()

In [ ]:
df5.drop(['Flavor'], axis=1,inplace=True)

In [ ]:
df5.isnull().sum()

In [ ]:
df6=pd.merge(df1,df3,on='RecipeId',how='inner')
df6

In [ ]:
df_time=df6[['SubmittedTime','Predicted_rating_x']].copy()

In [ ]:
df_time.rename(columns={'SubmittedTime':'ds','Predicted_rating_x':'y'},inplace=True)

In [ ]:
df_time.head()

In [ ]:
import matplotlib.pyplot as plt
from prophet.plot import add_changepoints_to_plot
from prophet import Prophet
m1 = Prophet()
m1.fit(df_time)
days = int(input('Enter the number of years'))
future = m1.make_future_dataframe(periods = days*365)
forecast = m1.predict(future)

fig1 = m1.plot(forecast)
fig2 = m1.plot_components(forecast)

plt.show()

In [ ]:
df5

In [ ]:
s='2 hr 15 min'
s1='10 min'
import re

result = re.sub('\D', ' ',s)

result=result.split(' ')

print(result)

More the number of calories, worse the reviews seem to be

In [ ]:
sns.heatmap(df5.corr(), cmap="YlGnBu", annot=True)

In [ ]:
import seaborn as sns
sns.barplot(y=df5['Calories'],x=df5['Predicted_rating'],palette='mako')

In [ ]:
sns.barplot(y=df5['NumberOfIngredients'],x=df5['Predicted_rating'],palette='mako')

In [ ]:
plt.figure(figsize=(30,15))
sns.barplot(x=df5['NumberOfIngredients'],y=df5['Calories'],hue=df5['Predicted_rating'],palette='mako',ci=None)

In [ ]:
plt.figure(figsize=(30,15))
sns.barplot(x=df5['Flavor_Type'],hue=df5['Predicted_rating'],y=df5['Calories'],palette='mako',ci=None)

In [ ]:
plt.figure(figsize=(30,15))
sns.barplot(x=df5['Meal_Type'],hue=df5['Predicted_rating'],y=df5['Calories'],palette='mako',ci=None)
plt.xticks(rotation=45)

In [ ]:
df6=df5.copy()

label encoding categorical data

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df6['Flavor_Type']=le.fit_transform(df6['Flavor_Type'])

In [ ]:
le=LabelEncoder()
df6['Dish']=le.fit_transform(df6['Dish'])

In [ ]:
le=LabelEncoder()
df6['Meal_Type']=le.fit_transform(df6['Meal_Type'])

In [ ]:
df6

Basic Machine Learning Model based on all attributes but the ingredients

In [ ]:
X=df6.loc[:, ['Flavor_Type','Dish','Calories','Meal_Type','NumberOfIngredients']]
y=df6['Predicted_rating']

In [ ]:
print(X)

In [ ]:
print(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
!pip install lazypredict

In [ ]:
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
df2

In [ ]:
r_unique=df2['RecipeId'].unique()
r_unique

Finding the difference between the maximum and minimum rating for a review

In [ ]:
diff=[]
for i in r_unique:
  l2=[]
  for j in range(len(df2)):
    try:
      if(df2['RecipeId'][j]==i):
        l2.append(df2['Predicted_rating'][j])
    except:
      pass
  difference=max(l2)-min(l2)
  diff.append(difference)

In [ ]:
print(diff)

In [ ]:
for i in range(9):
  diff.append(4)

In [ ]:
df6['diff']=diff

In [ ]:
plt.figure(figsize=(30,15))
sns.barplot(hue=df6['Calories'],x=df6['Predicted_rating'],y=df6['diff'],palette='mako',ci=None)